In [21]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

conn = psycopg2.connect(
                        host="127.0.0.1",
                        dbname="sparkifydb",
                        user="chaiwon" ,
                        password="chaiwon1"
                        )
cur = conn.cursor()
print("sparkifydb에 접속하였습니다.")


sparkifydb에 접속하였습니다.


<h3>파일 로드해서 각 테이블 drop, insert 하나씩 넣어보기</h3>

In [26]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

<h3>song_data load</h3>

In [27]:
song_files = get_files('data/song_data')
filepath = song_files[0]
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


<h4>songs 테이블</h4>

In [29]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [30]:
cur.execute(song_table_drop)
cur.execute(song_table_create)
cur.execute(song_table_insert, song_data)
conn.commit()

<h4>artists 테이블</h4>

In [33]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

In [34]:
cur.execute(artist_table_drop)
cur.execute(artist_table_create)
cur.execute(artist_table_insert, artist_data)
conn.commit()

<h3>log_data load</h3>

In [35]:
log_files = get_files('data/log_data')
filepath = log_files[0]
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


<h4>time 테이블</h4>

In [36]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [37]:
t = pd.to_datetime(df['ts'])
t.head()

0   1970-01-01 00:25:41.903636796
1   1970-01-01 00:25:41.903770796
2   1970-01-01 00:25:41.904034796
4   1970-01-01 00:25:41.910973796
5   1970-01-01 00:25:41.911006796
Name: ts, dtype: datetime64[ns]

In [38]:
time_data = [(tt.value, tt.hour, tt.day, tt.week, tt.month, tt.year, tt.weekday()) for tt in t]
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [39]:
time_df = pd.DataFrame(data=time_data, columns=column_labels)
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,1541903636796,0,1,1,1,1970,3
1,1541903770796,0,1,1,1,1970,3
2,1541904034796,0,1,1,1,1970,3
3,1541910973796,0,1,1,1,1970,3
4,1541911006796,0,1,1,1,1970,3


In [40]:
cur.execute(time_table_drop)
cur.execute(time_table_create)

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

<h4>users 테이블</h4>

In [41]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free


In [42]:
cur.execute(user_table_drop)
cur.execute(user_table_create)

for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

<h4>songplays 테이블</h4>

In [44]:
cur.execute(songplay_table_drop)
cur.execute(songplay_table_create)

In [45]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'],
                     row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [46]:
conn.close()